In [9]:
import sys
import os
import collections
import time
import datetime
import multiprocessing
import concurrent.futures
import numpy as np
import pandas as pd
import pickle
sys.path.append('/scratch/c/chutcher/wilsodj/MADE/')
import modules.utils_addm_007 as utils_addm     # for importing custom module
import importlib

In [58]:
importlib.reload(utils_addm)

<module 'modules.utils_addm_007' from '/scratch/c/chutcher/wilsodj/MADE/modules/utils_addm_007.py'>

In [2]:
# Version
version_num = "007/"
# Which node
node_num = 1
task = "01a_sim"

# Date
now = datetime.datetime.now().strftime('%Y_%m_%d_%H_')
print(version_num + now + task)

# Path
path = "MADE/" + "version/" + version_num + "output/" + now + "_" + task + "/"
# Make directory
os.makedirs(path, exist_ok=True)

007/2018_11_06_21_01a_sim


In [3]:
# Trial Data
expdata_file_name = "/scratch/c/chutcher/wilsodj/MADE/data/made_v2/expdata.csv"
# Fixation Data
fixations_file_name = "/scratch/c/chutcher/wilsodj/MADE/data/made_v2/fixations.csv"

In [4]:
# SCALING VALUE FOR DRIFT
drift_weight = np.linspace(0.005, 0.15, 16)
# BOUNDARY VALUE
upper_boundary = np.linspace(0.05, 0.35, 16)
# THETA VALUE
theta = np.linspace(0.1, 1, 12)
# START BIAS VALUE
sp_bias = np.linspace(-0.3, 0.3, 7)
parameter_combos = utils_addm.parameter_values(drift_weight, upper_boundary, theta, sp_bias)
# To save search space later
parameter_search_space = {"drift_weight": drift_weight,
                          "upper_boundary": upper_boundary,
                          "theta": theta,
                          "sp_bias": sp_bias}


#----------------#
# VALUES ARRAY   #
#----------------#
sims_per_val = 1000
# Get left and right values from data
values_array_addm = utils_addm.values_for_simulation_addm(expdata_file_name, sims_per_val)
# Create num_sims var
num_sims = values_array_addm.shape[1]

input_vals = utils_addm.Input_Values(
    parameter_combos=parameter_combos,
    values_array=values_array_addm,
    num_sims=num_sims,
    startVar=0,
    nonDec=0.3,
    nonDecVar=0,
    driftVar=0,
    maxRT=10,
    precision=0.001,        # ms precision
    s=0.1,                  # scaling factor (Ratcliff) (check if variance or SD)
)

Your parameter search space is size: (21504, 4).
You will run 1000 simulations for each of 81 value combinations for a total of 81000 simulations.


In [5]:
def rt_dist_sim(parameters):
    # NEXT: dwell_array would be by individual
    dwell_array = utils_addm.create_dwell_array(num_sims, fixations_file_name, data='sim')
    # Run simulations
    rtDist = utils_addm.simul_addm_rt_dist(parameters, input_vals, dwell_array, 'percent')

    return rtDist

In [62]:
x1 = rt_dist_sim(parameter_combos[0])